# Customer Lifetime Value (CLV) Prediction
## Year 1 Data- Feature Engineering & Dataset Preparation
We aim to predict **Customer Lifetime Value (CLV)** using customer purchase behavior observed in **Year 1**. The idea is to use historical transactional data to estimate a customer's future value.

---
### Dataset Overview:
We’re working with the `online_retail_II.xlsx` dataset, which contains **two sheets**:
- **Sheet 1** — Transactions from **2009-12-01 to 2010-12-09** (Year 1)
- **Sheet 2** — Transactions from **2010-12-01 to 2011-12-09** (Year 2)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [2]:
data = pd.read_excel("online_retail_II.xlsx",sheet_name=0)

In [ ]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
data['Customer ID']=data['Customer ID'].astype('Int64')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525461 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      525461 non-null  object        
 1   StockCode    525461 non-null  object        
 2   Description  522533 non-null  object        
 3   Quantity     525461 non-null  int64         
 4   InvoiceDate  525461 non-null  datetime64[ns]
 5   Price        525461 non-null  float64       
 6   Customer ID  417534 non-null  Int64         
 7   Country      525461 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 32.6+ MB


---
### Data Cleaning & Filtering
---
#### Drop rows with missing `Customer ID`

In [ ]:
data.dropna(subset=['Customer ID'], inplace=True)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417534 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      417534 non-null  object        
 1   StockCode    417534 non-null  object        
 2   Description  417534 non-null  object        
 3   Quantity     417534 non-null  int64         
 4   InvoiceDate  417534 non-null  datetime64[ns]
 5   Price        417534 non-null  float64       
 6   Customer ID  417534 non-null  Int64         
 7   Country      417534 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 29.1+ MB


#### Filtering based on StockCode
- M, S, SPxxxx, TESTxxxx -> will drop 
- Gifts, PADS, POST -> will keep
**NOTE:**
    - `PADS`-> is a valid item
    - `POST`-> Postage service(add value to CLV)

In [ ]:
# Handling Gifts
filtered_data = data[data['StockCode'].str.contains("gift", case=False, na=False)]
filtered_data  

# Looks like no rows with gift left...

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


In [ ]:
unwanted_codes = ['M'] + [
    code for code in data['StockCode'].unique()
    if any(x in str(code).upper() for x in ['TEST', 'GIFT', 'SP'])
]
data = data[~data['StockCode'].isin(unwanted_codes)]


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 416866 entries, 0 to 525460
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      416866 non-null  object        
 1   StockCode    416866 non-null  object        
 2   Description  416866 non-null  object        
 3   Quantity     416866 non-null  int64         
 4   InvoiceDate  416866 non-null  datetime64[ns]
 5   Price        416866 non-null  float64       
 6   Customer ID  416866 non-null  Int64         
 7   Country      416866 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 29.0+ MB


#### Remove Returened Invoices (Credit Back, Invoices as 'Cxxxxxx')

In [ ]:
data =data[~data['Invoice'].astype(str).str.startswith('C')]
data['Invoice']=data['Invoice'].astype('int64')

#### Remove negative or zero quantity/price

In [ ]:
data=data[(data['Quantity']>0) & (data['Price']>0)]

### Handling Overlap in Year Splits
**Upon inspecting the dataset, we observed that:**
- Year 1 ends on: 09-Dec-2010 20:01
- Year 2 starts on: 01-Dec-2010 08:26

This reveals a 9-day overlap, which poses a risk of data leakage **to resolve this, we will:**
- Define Year 1 as all transactions `'before' 01-Dec-2010`
- Define Year 2 as all transactions from `01-Dec-2010 'onward'`

In [ ]:
year1_end = pd.to_datetime("2010-11-30 23:59:59")
data = data[data['InvoiceDate'] <= year1_end].copy()
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392741 entries, 0 to 502937
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      392741 non-null  int64         
 1   StockCode    392741 non-null  object        
 2   Description  392741 non-null  object        
 3   Quantity     392741 non-null  int64         
 4   InvoiceDate  392741 non-null  datetime64[ns]
 5   Price        392741 non-null  float64       
 6   Customer ID  392741 non-null  Int64         
 7   Country      392741 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 27.3+ MB


In [ ]:
data.shape

(392741, 8)

---
### Feature Engineering:
**Engineer behavioral features** for each customer, including:
   - Total Spend
   - Purchase Frequency
   - Average Quantity per Invoice (Order Habit)
   - Recency (in days)
   - Average Purchase Gap (time between purchases)
   - Buyer Type (Bulk/Light)
   - Trend in Spending (via 6-Quarter breakdown)
   - Churn Flag

---

**Feature 1:** **`Total Spend`**
It’s the sum of money a customer has spent in Year 1.<br>
***`Total Spend = Sum(Quantity × Price)`***<br>

For this feature valid transaction are:
- Qty > 0
- Price > 0
- CustomerId Not NuLL
- Unwanted StockCode: like, Test, Sp, M, etc...(removed)<br>
(which is already done!)

In [ ]:
data['TotalPrice']= data['Quantity']*data['Price']
total_spend = data.groupby('Customer ID')['TotalPrice'].sum().reset_index()
total_spend

,Customer ID,TotalPrice
0,12346,169.36
1,12347,611.53
2,12348,222.16
3,12349,2671.14
4,12351,300.93
...,...,...
4247,18283,641.77
4248,18284,461.68
4249,18285,427.00
4250,18286,1296.43


In [ ]:
total_spend.columns = ['Customer ID', 'TotalSpend']
features_df = total_spend.copy()
features_df

,Customer ID,TotalSpend
0,12346,169.36
1,12347,611.53
2,12348,222.16
3,12349,2671.14
4,12351,300.93
...,...,...
4247,18283,641.77
4248,18284,461.68
4249,18285,427.00
4250,18286,1296.43


**Feature 2:** **`PurchaseFrequency`**<br>
***`Purchase Frequency = distinct(Invoices) a customer placed in Year 1.`***
- Each Invoice represents one purchase event.
- We’ll count how many unique Invoice numbers are associated with each Customer ID.

In [ ]:
Frequency= data.groupby('Customer ID')['Invoice'].nunique().reset_index()
Frequency.columns=['Customer ID', 'PurchaseFrequency']
features_df = features_df.merge(Frequency, on='Customer ID', how='left')

In [ ]:
features_df.sample(10)

,Customer ID,TotalSpend,PurchaseFrequency
296,12817,524.94,2
1602,14627,462.75,1
2638,16071,750.56,3
3241,16907,544.23,4
2292,15591,217.90,3
1443,14421,1656.26,4
326,12859,575.10,1
2777,16258,6739.01,10
3132,16755,1403.75,4
754,13465,931.12,2


**Feature 3:** **`OrderHabit`**<br>
***`Order Habit = Sum(Quantity)/ Unique(Invoices)`***<br>
The Order Habit feature tells us how many items a customer typically buys per invoice.
*This helps us understand:*
- Whether the customer buys small or large amounts in one go.
- Their typical purchasing behavior.
    - bulk
    - light

In [ ]:
total_qty=data.groupby('Customer ID')['Quantity'].sum().reset_index()
total_qty.columns=['Customer ID', 'TotalQty']
orderHabit=Frequency.merge(total_qty, on='Customer ID')
orderHabit['OrderHabit']=orderHabit['TotalQty']/orderHabit['PurchaseFrequency']
features_df = features_df.merge(orderHabit[['Customer ID', 'OrderHabit']], on='Customer ID', how='left')

In [ ]:
features_df.sample(10)

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit
3025,16600,1339.89,7,182.142857
3352,17062,375.42,2,117.000000
265,12778,953.45,3,303.333333
3152,16782,7786.81,25,139.360000
2428,15786,3582.62,2,1308.500000
3174,16817,955.95,4,144.750000
3486,17235,1012.92,6,103.500000
0,12346,169.36,2,12.000000
2079,15283,148.33,3,9.000000
449,13031,3419.27,5,479.800000


**Feature 4:** **`Recency`**:How recently a customer made their last purchase<br>
***`Recency = Reference Date - Last Purchase Date`***<br>
It helps answer:
*“Is this customer still active?”*

In [ ]:
cutoff_date = pd.to_datetime("2010-12-1")
recency=data.groupby('Customer ID')['InvoiceDate'].max().reset_index()
recency.columns=['Customer ID','LastPurchase']
recency['Recency']=(cutoff_date-recency['LastPurchase']).dt.days

In [ ]:
features_df=features_df.merge(recency[['Customer ID', 'Recency']], on='Customer ID', how='left')
features_df

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency
0,12346,169.36,2,12.00,155
1,12347,611.53,1,509.00,30
2,12348,222.16,1,373.00,64
3,12349,2671.14,3,331.00,33
4,12351,300.93,1,261.00,1
...,...,...,...,...,...
4247,18283,641.77,6,56.00,8
4248,18284,461.68,1,494.00,57
4249,18285,427.00,1,145.00,286
4250,18286,1296.43,2,304.00,102


**Feature 5:** **`AvgPurchaseGap`**:Average Time Between Purchases<br>
A shorter interval indicates a more engaged or loyal customer.

In [ ]:
sortedData=data.sort_values(by=['Customer ID', 'InvoiceDate'])

sortedData['prev']=sortedData.groupby('Customer ID')['InvoiceDate'].shift(1)
sortedData['diff']=(sortedData['InvoiceDate'] - sortedData['prev']).dt.days

intervals=sortedData.dropna(subset=['diff'])

In [ ]:
purchase_interval=intervals.groupby('Customer ID')['diff'].mean().reset_index()
purchase_interval.columns=['Customer ID', 'AvgPurchaseGap']

In [ ]:
features_df= features_df.merge(purchase_interval, on='Customer ID', how='left')

In [ ]:
features_df.sample(10)

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap
3354,17065,194.70,2,41.000000,263,0.000000
1329,14254,601.03,1,263.000000,58,0.000000
907,13669,195.00,1,156.000000,33,0.000000
27,12385,1938.40,1,612.000000,5,0.000000
1528,14532,2926.33,7,118.714286,37,1.126482
589,13214,3264.23,7,423.714286,43,0.991228
1260,14167,721.65,4,50.000000,90,3.161765
2872,16385,326.49,2,109.000000,15,4.321429
2973,16530,48.35,1,17.000000,294,0.000000
889,13648,1764.02,8,120.000000,26,3.180952


In [ ]:
# Define cutoff date
cutoff_date = pd.to_datetime("2010-12-1")

# Calculate first purchase per customer
first_purchase = data.groupby('Customer ID')['InvoiceDate'].min().reset_index()
first_purchase.columns = ['Customer ID', 'FirstPurchase']
first_purchase['Tenure'] = (cutoff_date - first_purchase['FirstPurchase']).dt.days

# Merge tenure into your feature dataframe
features_df = features_df.merge(first_purchase[['Customer ID', 'Tenure']], on='Customer ID', how='left')


In [ ]:
features_df.sample(10)

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap,Tenure
2005,15183,639.93,3,99.000000,173,0.972973,210
1542,14550,587.69,2,191.000000,14,1.938776,109
1362,14299,10104.63,18,332.611111,13,1.801047,363
2227,15496,146.14,1,98.000000,257,0.000000,257
1963,15124,266.63,2,114.500000,102,11.666667,277
2149,15382,7777.84,6,861.000000,0,1.091185,361
950,13728,1839.03,10,81.100000,13,3.370000,356
309,12839,4077.70,12,230.916667,5,1.456067,358
2828,16325,3003.26,9,252.222222,50,2.571429,361
3042,16628,130.05,2,27.500000,33,33.285714,267


**Feature 6:** **`Trend`**:Customer Spending Behavior Over Time<br>
- Increasing spending(`+1`)
- Decreasing spending(`-1`)
- Stable(`0`)

In [ ]:
data['total']=data['Quantity']*data['Price']

In [ ]:
year1_start = pd.to_datetime("2009-12-01")
quarter_dates=[
    (year1_start,year1_start + pd.DateOffset(months=3)),
    (year1_start+pd.DateOffset(months=3), year1_start+pd.DateOffset(months=6)),
    (year1_start+pd.DateOffset(months=6), year1_start+pd.DateOffset(months=9)),
    (year1_start+pd.DateOffset(months=9), year1_start+pd.DateOffset(months=12))
]

In [ ]:
def assign_quarter(date):
    for i, (start,end) in enumerate(quarter_dates):
        if start<=date<end:
            return i+1
    return np.nan

data['Quarter']=data['InvoiceDate'].apply(assign_quarter)

In [ ]:
data.sample(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,total,Quarter
45394,493435,22028,PENNY FARTHING BIRTHDAY CARD,12,2010-01-04 12:57:00,0.42,13206,United Kingdom,5.04,5.04,1
112339,500129,84461,12 PINK HEN+CHICKS IN BASKET,1,2010-03-04 14:50:00,2.55,15768,United Kingdom,2.55,2.55,2
158395,504478,20725,LUNCH BAG RED SPOTTY,10,2010-04-14 10:13:00,1.65,12682,France,16.50,16.50,2
410942,528694,21741,COSY SLIPPER SHOES LARGE GREEN,1,2010-10-24 13:01:00,2.95,15059,United Kingdom,2.95,2.95,4
11289,490299,85231L,LAVENDER SCENTED SET/9 T-LIGHTS,1,2009-12-04 14:14:00,0.85,13680,United Kingdom,0.85,0.85,1
11043,490292,21239,PINK SPOTTY CUP,1,2009-12-04 13:23:00,0.85,16723,United Kingdom,0.85,0.85,1
407794,528414,22959,WRAP CHRISTMAS VILLAGE,25,2010-10-22 09:31:00,0.42,12960,United Kingdom,10.50,10.50,4
447153,531971,22549,PICTURE DOMINOES,24,2010-11-10 12:24:00,1.45,13464,United Kingdom,34.80,34.80,4
204239,509115,22444,GROW YOUR OWN PLANT IN A CAN,24,2010-05-20 11:55:00,1.25,17716,United Kingdom,30.00,30.00,2
379855,525967,21992,VINTAGE PAISLEY STATIONERY SET,24,2010-10-08 10:00:00,1.25,14243,United Kingdom,30.00,30.00,4


In [ ]:
quarterly_spend = data.groupby(['Customer ID', 'Quarter'])['total'].sum().unstack(fill_value=0)

In [ ]:
quarterly_spend.head()

Quarter,1,2,3,4
Customer ID,,,,
12346,0.0,27.05,142.31,0.00
12347,0.0,0.00,0.00,611.53
12348,0.0,0.00,0.00,222.16
12349,0.0,1268.52,0.00,1402.62
12351,0.0,0.00,0.00,300.93


In [ ]:
quarterly_spend.columns=[f'Q{i}' for i in range(1,5)]

In [ ]:
quarterly_spend.head(10)

,Q1,Q2,Q3,Q4
Customer ID,,,,
12346,0.0,27.05,142.31,0.00
12347,0.0,0.00,0.00,611.53
12348,0.0,0.00,0.00,222.16
12349,0.0,1268.52,0.00,1402.62
12351,0.0,0.00,0.00,300.93
12352,0.0,0.00,0.00,343.80
12353,0.0,0.00,0.00,317.76
12355,0.0,488.21,0.00,0.00
12356,0.0,0.00,0.00,3562.25


In [ ]:
#lets chcek for 6-quarters
year1_start = pd.to_datetime("2009-12-01")
quarter_dates_6=[
    (year1_start,year1_start + pd.DateOffset(months=2)),
    (year1_start+pd.DateOffset(months=2), year1_start+pd.DateOffset(months=4)),
    (year1_start+pd.DateOffset(months=4), year1_start+pd.DateOffset(months=6)),
    (year1_start+pd.DateOffset(months=6), year1_start+pd.DateOffset(months=8)),
    (year1_start+pd.DateOffset(months=8), year1_start+pd.DateOffset(months=10)),
    (year1_start+pd.DateOffset(months=10), year1_start+pd.DateOffset(months=12))
]

In [ ]:
def assign_quarter(date):
    for i, (start,end) in enumerate(quarter_dates_6):
        if start<=date<end:
            return i+1
    return np.nan
data['Quarter_6']=data['InvoiceDate'].apply(assign_quarter)

In [ ]:
quarterly_spend_6 = data.groupby(['Customer ID', 'Quarter_6'])['total'].sum().unstack(fill_value=0)

In [ ]:
quarterly_spend_6.columns=[f'Q{i}' for i in range(1,7)]

In [ ]:
quarterly_spend_6.head()

,Q1,Q2,Q3,Q4,Q5,Q6
Customer ID,,,,,,
12346,0.0,27.05,0.00,142.31,0.00,0.00
12347,0.0,0.00,0.00,0.00,0.00,611.53
12348,0.0,0.00,0.00,0.00,222.16,0.00
12349,0.0,0.00,1268.52,0.00,0.00,1402.62
12351,0.0,0.00,0.00,0.00,0.00,300.93


***Will use 6- Quarter for trend analysis and Churn Finding***

In [ ]:
trend_results = []

for index, row in quarterly_spend_6.iterrows():
    customer_id = index
    spends = row[['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6']].values.astype(float)

    non_zero_periods = np.count_nonzero(spends)
    active_quarters = np.where(spends > 0)[0]  

    if non_zero_periods == 1 and active_quarters[0] == 5:
        trend_results.append((customer_id, 'Flag')) 
        continue

    x = np.arange(1, 7)
    slope, intercept = np.polyfit(x, spends, 1)

    if slope > 0.5:
        trend = 'Increasing'
    elif slope < -0.5:
        trend = 'Decreasing'
    else:
        trend = 'Stable'

    trend_results.append((customer_id, trend))

In [ ]:
trend_df = pd.DataFrame(trend_results, columns=['Customer ID', 'Trend'])
features_df = features_df.merge(trend_df, on='Customer ID', how='left')

In [ ]:
features_df

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap,Tenure,Trend
0,12346,169.36,2,12.00,155,5.130435,273,Increasing
1,12347,611.53,1,509.00,30,0.000000,30,Flag
2,12348,222.16,1,373.00,64,0.000000,64,Increasing
3,12349,2671.14,3,331.00,33,1.782178,215,Increasing
4,12351,300.93,1,261.00,1,0.000000,1,Flag
...,...,...,...,...,...,...,...,...
4247,18283,641.77,6,56.00,8,1.192140,284,Increasing
4248,18284,461.68,1,494.00,57,0.000000,57,Flag
4249,18285,427.00,1,145.00,286,0.000000,286,Decreasing
4250,18286,1296.43,2,304.00,102,3.742424,349,Increasing


In [ ]:
features_df['Trend'].unique()

array(['Increasing', 'Flag', 'Decreasing', 'Stable'], dtype=object)

In [ ]:
features_df['Trend'].value_counts()

Trend
Increasing    2052
Decreasing    1469
Flag           701
Stable          30
Name: count, dtype: int64

In [ ]:
features_df

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap,Tenure,Trend
0,12346,169.36,2,12.00,155,5.130435,273,Increasing
1,12347,611.53,1,509.00,30,0.000000,30,Flag
2,12348,222.16,1,373.00,64,0.000000,64,Increasing
3,12349,2671.14,3,331.00,33,1.782178,215,Increasing
4,12351,300.93,1,261.00,1,0.000000,1,Flag
...,...,...,...,...,...,...,...,...
4247,18283,641.77,6,56.00,8,1.192140,284,Increasing
4248,18284,461.68,1,494.00,57,0.000000,57,Flag
4249,18285,427.00,1,145.00,286,0.000000,286,Decreasing
4250,18286,1296.43,2,304.00,102,3.742424,349,Increasing


In [ ]:
quarterly_spend_6=quarterly_spend_6.merge(trend_df, on='Customer ID', how='left')

In [ ]:
quarterly_spend_6

,Customer ID,Q1,Q2,Q3,Q4,Q5,Q6,Trend
0,12346,0.00,27.05,0.00,142.31,0.00,0.00,Increasing
1,12347,0.00,0.00,0.00,0.00,0.00,611.53,Flag
2,12348,0.00,0.00,0.00,0.00,222.16,0.00,Increasing
3,12349,0.00,0.00,1268.52,0.00,0.00,1402.62,Increasing
4,12351,0.00,0.00,0.00,0.00,0.00,300.93,Flag
...,...,...,...,...,...,...,...,...
4247,18283,0.00,354.42,0.00,0.00,92.00,195.35,Increasing
4248,18284,0.00,0.00,0.00,0.00,0.00,461.68,Flag
4249,18285,0.00,427.00,0.00,0.00,0.00,0.00,Decreasing
4250,18286,462.95,0.00,0.00,0.00,833.48,0.00,Increasing


In [ ]:
increasing_df = quarterly_spend_6[quarterly_spend_6['Trend'] == 'Increasing']
increasing_df

,Customer ID,Q1,Q2,Q3,Q4,Q5,Q6,Trend
0,12346,0.00,27.05,0.00,142.31,0.00,0.00,Increasing
2,12348,0.00,0.00,0.00,0.00,222.16,0.00,Increasing
3,12349,0.00,0.00,1268.52,0.00,0.00,1402.62,Increasing
12,12360,0.00,158.00,622.04,0.00,0.00,810.79,Increasing
17,12369,0.00,348.32,0.00,0.00,1259.48,183.35,Increasing
...,...,...,...,...,...,...,...,...
4242,18277,0.00,0.00,0.00,0.00,337.14,732.53,Increasing
4244,18279,0.00,0.00,0.00,231.34,0.00,0.00,Increasing
4247,18283,0.00,354.42,0.00,0.00,92.00,195.35,Increasing
4250,18286,462.95,0.00,0.00,0.00,833.48,0.00,Increasing


**Feature 7:** **`Churn`**:
Binary indicator where `1` denotes customer inactivity in the last two quarters (i.e., no purchases), suggesting a potential churn. This serves as a behavioral flag to signal declining engagement.

In [ ]:
def is_churn(row):
    early_active = any(row[['Q1', 'Q2', 'Q3', 'Q4']].astype(float) > 0)
    last_two_inactive = float(row['Q5']) == 0 and float(row['Q6']) == 0
    return int(early_active and last_two_inactive)

quarterly_spend_6['Churn'] = quarterly_spend_6.apply(is_churn, axis=1)

In [ ]:
churn_df = quarterly_spend_6.reset_index()[['Customer ID', 'Churn']]

In [ ]:
quarterly_spend_6.sample(10)

,Customer ID,Q1,Q2,Q3,Q4,Q5,Q6,Trend,Churn
1444,14422,675.46,805.72,949.32,954.35,1597.77,1025.26,Increasing,0
1757,14839,425.06,0.00,184.02,0.00,0.00,306.20,Decreasing,0
1667,14713,349.23,229.71,458.15,385.56,243.12,787.39,Increasing,0
457,13041,0.00,0.00,256.16,350.92,192.05,0.00,Increasing,0
2509,15901,0.00,0.00,0.00,203.69,117.80,0.00,Increasing,0
473,13063,0.00,0.00,0.00,0.00,188.40,0.00,Increasing,0
1302,14217,0.00,0.00,0.00,416.60,272.48,604.53,Increasing,0
310,12840,0.00,150.00,0.00,491.69,673.51,0.00,Increasing,0
2595,16012,0.00,0.00,0.00,975.33,0.00,599.17,Increasing,0
2865,16374,0.00,0.00,473.89,0.00,434.09,384.54,Increasing,0


In [ ]:
features_df = features_df.merge(churn_df, on='Customer ID', how='left')

In [ ]:
features_df.sample(10)

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap,Tenure,Trend,Churn
1450,14430,1220.90,3,148.666667,137,2.794118,233,Decreasing,1
338,12871,363.38,1,284.000000,65,0.000000,65,Increasing,0
2196,15456,1672.05,7,309.428571,29,1.923077,357,Decreasing,0
3836,17700,4145.33,10,220.300000,7,1.631336,364,Decreasing,0
3997,17924,295.20,1,48.000000,40,0.000000,40,Flag,0
2576,15989,372.51,2,85.500000,1,4.149425,363,Increasing,0
2059,15259,1182.38,4,157.750000,138,3.017857,308,Decreasing,1
1655,14696,394.33,1,237.000000,47,0.000000,47,Flag,0
1368,14307,1292.20,4,189.500000,134,2.740741,358,Decreasing,1
3233,16896,103.95,1,18.000000,181,0.000000,181,Increasing,1


**Feature 8:** **`ReturnRate`**<br>
***`return_rate = abs(ReturnQty) / (ReturnQty + PurchaseQty)`***<br>
Return Rate measures how often a customer returns products. A high return rate could indicate dissatisfaction or exploitative behavior.

In [ ]:
returned_data = pd.read_excel("online_retail_II.xlsx",sheet_name=0)

In [ ]:
returned_data = returned_data[returned_data['Invoice'].astype(str).str.startswith('C')].copy()

In [ ]:
# `data` has Purchase
# `returned_data` has return data

In [ ]:
purchase_qty = data.groupby('Customer ID')['Quantity'].sum().reset_index()
purchase_qty.columns = ['Customer ID', 'TotalQtyPurchased']

In [ ]:
return_qty = returned_data.groupby('Customer ID')['Quantity'].sum().abs().reset_index()
return_qty.columns = ['Customer ID', 'TotalQtyReturned']

In [ ]:
purchase_qty['Customer ID'] = purchase_qty['Customer ID'].astype('int64')
return_qty['Customer ID'] = return_qty['Customer ID'].astype('int64')

In [ ]:
return_rate_df = purchase_qty.merge(return_qty, on='Customer ID', how='left')
return_rate_df['TotalQtyReturned'].fillna(0, inplace=True)

In [ ]:
return_rate_df.head()

,Customer ID,TotalQtyPurchased,TotalQtyReturned
0,12346,24,18.0
1,12347,509,0.0
2,12348,373,0.0
3,12349,993,5.0
4,12351,261,0.0


In [ ]:
return_rate_df['ReturnRate'] = return_rate_df['TotalQtyReturned'] / (
    return_rate_df['TotalQtyPurchased'] + return_rate_df['TotalQtyReturned']
)

In [ ]:
return_rate_df.head()

,Customer ID,TotalQtyPurchased,TotalQtyReturned,ReturnRate
0,12346,24,18.0,0.428571
1,12347,509,0.0,0.000000
2,12348,373,0.0,0.000000
3,12349,993,5.0,0.005010
4,12351,261,0.0,0.000000


In [ ]:
features_df = features_df.merge(return_rate_df[['Customer ID', 'ReturnRate']], on='Customer ID', how='left')

In [ ]:
features_df.sample(10)

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap,Tenure,Trend,Churn,ReturnRate
990,13782,881.72,7,39.714286,79,2.164062,359,Decreasing,0,0.003584
1001,13795,836.30,4,312.500000,14,8.821429,263,Increasing,0,0.000000
3397,17121,314.74,1,106.000000,28,0.000000,28,Flag,0,0.000000
1558,14570,295.19,1,269.000000,69,0.000000,69,Increasing,0,0.000000
336,12869,337.67,1,144.000000,307,0.000000,307,Decreasing,1,0.000000
3621,17402,2002.36,10,75.000000,6,2.848739,349,Increasing,0,0.005305
2440,15800,543.50,5,50.600000,79,1.330645,244,Decreasing,0,0.000000
2612,16037,654.32,2,169.000000,170,3.322581,274,Decreasing,1,0.005882
633,13282,181.05,1,103.000000,32,0.000000,32,Flag,0,0.000000
322,12854,1669.52,4,252.000000,187,1.077922,354,Decreasing,1,0.015625


## Year 2 Data — Creating the Target Variable (CLV)
Now that we have successfully engineered 8 behavioral features from **Year 1 (Sheet 1)**, we shift our focus to **Year 2 (Sheet 2)** to create the **target variable** for our supervised learning task.

In [ ]:
data_y2= pd.read_excel("online_retail_II.xlsx",sheet_name=1)

In [ ]:
data_y2 = data_y2.dropna(subset=['Customer ID'])

In [ ]:
data_y2['Customer ID']=data_y2['Customer ID'].astype('Int64')

In [ ]:
data_y2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406830 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      406830 non-null  object        
 1   StockCode    406830 non-null  object        
 2   Description  406830 non-null  object        
 3   Quantity     406830 non-null  int64         
 4   InvoiceDate  406830 non-null  datetime64[ns]
 5   Price        406830 non-null  float64       
 6   Customer ID  406830 non-null  Int64         
 7   Country      406830 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 28.3+ MB


In [ ]:
data_y2 =data_y2[~data_y2['Invoice'].astype(str).str.startswith('C')]
data_y2['Invoice']=data_y2['Invoice'].astype('int64')

In [ ]:
filtered_data_y2 = data_y2[data_y2['StockCode'].str.contains("gift", case=False, na=False)]
filtered_data_y2  

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


In [ ]:
unwanted_codes_y2 = ['M'] + [
    code for code in data_y2['StockCode'].unique()
    if any(x in str(code).upper() for x in ['TEST', 'GIFT', 'SP'])
]
data_y2 = data_y2[~data_y2['StockCode'].isin(unwanted_codes)]

In [ ]:
data_y2=data_y2[(data_y2['Quantity']>0) & (data_y2['Price']>0)]

In [ ]:
year2_end = pd.to_datetime("2011-11-30 23:59:59")
data_y2 = data_y2[data_y2['InvoiceDate'] <= year2_end].copy()
data_y2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [ ]:
data_y2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 380304 entries, 0 to 516368
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      380304 non-null  int64         
 1   StockCode    380304 non-null  object        
 2   Description  380304 non-null  object        
 3   Quantity     380304 non-null  int64         
 4   InvoiceDate  380304 non-null  datetime64[ns]
 5   Price        380304 non-null  float64       
 6   Customer ID  380304 non-null  Int64         
 7   Country      380304 non-null  object        
dtypes: Int64(1), datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 26.5+ MB


In [ ]:
print("Valid Rows:")
print("Year1 shape",data.shape)
print("Year2 shape",data_y2.shape)

Valid Rows:
Year1 shape (392741, 12)
Year2 shape (380304, 8)


In [ ]:
data_y2['TotalSpend_Y2'] = data_y2['Quantity'] * data_y2['Price']

clv_y2 = data_y2.groupby('Customer ID')['TotalSpend_Y2'].sum().reset_index()
clv_y2.rename(columns={'TotalSpend_Y2': 'CLV'}, inplace=True)

In [ ]:
clv_y2.head()

,Customer ID,CLV
0,12346,77183.60
1,12347,4085.18
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


### Finaly merging with year 1:

In [ ]:
features_df_clv = features_df.merge(clv_y2, on='Customer ID', how='left')

In [ ]:
features_df_clv.head()

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap,Tenure,Trend,Churn,ReturnRate,CLV
0,12346,169.36,2,12.0,155,5.130435,273,Increasing,1,0.428571,77183.60
1,12347,611.53,1,509.0,30,0.000000,30,Flag,0,0.000000,4085.18
2,12348,222.16,1,373.0,64,0.000000,64,Increasing,0,0.000000,1797.24
3,12349,2671.14,3,331.0,33,1.782178,215,Increasing,0,0.005010,1757.55
4,12351,300.93,1,261.0,1,0.000000,1,Flag,0,0.000000,NaN


In [ ]:
features_df_clv.shape

(4252, 11)

In [ ]:
(~(features_df_clv['CLV'].isna())).sum()

2711

In [ ]:
final_features_df = features_df_clv.dropna(subset=['CLV'])

In [ ]:
final_features_df.shape

(2711, 11)

In [ ]:
final_features_df.tail()

,Customer ID,TotalSpend,PurchaseFrequency,OrderHabit,Recency,AvgPurchaseGap,Tenure,Trend,Churn,ReturnRate,CLV
4243,18278,240.30,1,74.00,30,0.000000,30,Flag,0,0.000000,173.90
4245,18280,307.55,1,149.00,20,0.000000,20,Flag,0,0.013245,180.60
4246,18281,120.32,1,92.00,203,0.000000,203,Decreasing,1,0.000000,80.82
4247,18283,641.77,6,56.00,8,1.192140,284,Increasing,0,0.000000,1880.93
4251,18287,2345.71,4,356.75,8,2.238095,197,Increasing,0,0.001400,1837.28


In [ ]:
final_features_df.to_csv('final_clv_dataset.csv', index=False)

## Finally we are DONE!!, from raw data to a clean, feature-rich dataset ready for modeling.
### ***Will continue modeling in new notebook***